# Model 7: Batch Size Part 2

This notebook contains the construction and training of the model iterations and various experiments. The notebook is split up into four sections: training mode selection (where the model will run), set-up, model constrution, and training. 

Evaluation will take place in the *model_optimization_and_evaluation.ipynb* notebook found in the *notebooks* folder.

In [1]:
# importing necessary packages and libraries
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import optimizers
from keras import applications 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import pickle

Using TensorFlow backend.


## Step 1: Training Mode Selection

In the cell below, specify the training mode for the model. This will determine the location from which the source data is drawn, and to which the trained models (and training histories) are saved. 

- **training_mode = 'floydhub'** (runs on Floydhub)
- **training_mode = 'local'** (runs on local disk and processor)

In [2]:
# select training mode
training_mode = 'floydhub'

## Step 2: Set-up

In [3]:
# directory base paths
data_path_local = '../../data/0002_array_data/train_data/'
model_path_local = '../../notebooks/model_construction/saved_models/'
data_path_floydhub = '/floyd/input/capstone_mushrooms/'
model_path_floydhub = '/floyd/home/'

# setting directory paths based on training mode selection
if training_mode == 'floydhub':
    data_path = data_path_floydhub
    model_path = model_path_floydhub
elif training_mode == 'local':
    data_path = data_path_local
    model_path = model_path_local
else:
    raise Exception('Please choose valid training mode: "floydhub" or "local".')

In [4]:
# loading the training and validation data subsets
X_train = np.load(f'{data_path}X_train_data.npy')
y_train = np.load(f'{data_path}y_train_data.npy')
X_val = np.load(f'{data_path}X_val_data.npy')
y_val = np.load(f'{data_path}y_val_data.npy')

In [5]:
# setting training parameters
n_classes = 20
n_epochs = 30
img_shape = X_train.shape[1:]
model_names = []
model_list = []
model_hists = []

## Step 3: Model Construction

### Trial 1

In [6]:
# defining the model architecture
m7_t1 = models.Sequential()

# convolution/max pool stacks
m7_t1.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same'))
m7_t1.add(layers.LeakyReLU(alpha=0.1))
m7_t1.add(layers.MaxPooling2D((2,2)))

m7_t1.add(layers.Conv2D(64,(3,3), padding='same'))
m7_t1.add(layers.LeakyReLU(alpha=0.1))
m7_t1.add(layers.MaxPooling2D((2,2)))

m7_t1.add(layers.Conv2D(128,(3,3), padding='same'))
m7_t1.add(layers.LeakyReLU(alpha=0.1))
m7_t1.add(layers.MaxPooling2D((2,2)))

m7_t1.add(layers.Conv2D(256,(3,3), padding='same'))
m7_t1.add(layers.LeakyReLU(alpha=0.1))
m7_t1.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m7_t1.add(layers.Flatten())
m7_t1.add(layers.Dense(512))
m7_t1.add(layers.LeakyReLU(alpha=0.1))
m7_t1.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m7_t1.summary()
model_names.append('m7_t1')
model_list.append(m7_t1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 200, 200, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 128)      

### Trial 2

In [7]:
# defining the model architecture
m7_t2 = models.Sequential()

# convolution/max pool stacks
m7_t2.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same'))
m7_t2.add(layers.LeakyReLU(alpha=0.1))
m7_t2.add(layers.MaxPooling2D((2,2)))

m7_t2.add(layers.Conv2D(64,(3,3), padding='same'))
m7_t2.add(layers.LeakyReLU(alpha=0.1))
m7_t2.add(layers.MaxPooling2D((2,2)))

m7_t2.add(layers.Conv2D(128,(3,3), padding='same'))
m7_t2.add(layers.LeakyReLU(alpha=0.1))
m7_t2.add(layers.MaxPooling2D((2,2)))

m7_t2.add(layers.Conv2D(256,(3,3), padding='same'))
m7_t2.add(layers.LeakyReLU(alpha=0.1))
m7_t2.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m7_t2.add(layers.Flatten())
m7_t2.add(layers.Dense(512))
m7_t2.add(layers.LeakyReLU(alpha=0.1))
m7_t2.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m7_t2.summary()
model_names.append('m7_t2')
model_list.append(m7_t2)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 200, 200, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 50, 50, 128)      

## Step 4: Training

In [8]:
# setting up standardization and augmentation parameters
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   fill_mode='nearest',
                                   horizontal_flip=True,
                                   vertical_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
%%time
# data standardization and augmentation
train_generator_1 = train_datagen.flow(X_train, y_train, batch_size=4)
train_generator_2 = train_datagen.flow(X_train, y_train, batch_size=2)
val_generator_1 = val_datagen.flow(X_val, y_val, batch_size=4)
val_generator_2 = val_datagen.flow(X_val, y_val, batch_size=2)

CPU times: user 763 ms, sys: 1.91 s, total: 2.67 s
Wall time: 2.67 s


In [10]:
# setting up optimizer
opt = optimizers.Adamax(learning_rate=1e-3)

# compiling loss functions
m7_t1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc'])
m7_t2.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc'])

In [11]:
%%time
# setting up model saving checkpoints
m7_t1_cp = ModelCheckpoint(filepath=f'{model_path}m7_t1.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m7_t1_history = m7_t1.fit(train_generator_1,
                                steps_per_epoch=len(X_train)//4,
                                epochs=n_epochs,
                                callbacks=[m7_t1_cp],
                                validation_data=val_generator_1,
                                validation_steps=len(X_val)//4)

# adding training history to list
model_hists.append(m7_t1_history)

Epoch 1/30
2027/2027 [==============================] - 106s 52ms/step - loss: 2.5904 - acc: 0.1830 - val_loss: 2.3221 - val_acc: 0.3082

Epoch 00001: val_acc improved from -inf to 0.30817, saving model to /floyd/home/m7_t1.h5
Epoch 2/30
2027/2027 [==============================] - 79s 39ms/step - loss: 2.2449 - acc: 0.2796 - val_loss: 1.5651 - val_acc: 0.3759

Epoch 00002: val_acc improved from 0.30817 to 0.37593, saving model to /floyd/home/m7_t1.h5
Epoch 3/30
2027/2027 [==============================] - 79s 39ms/step - loss: 2.1139 - acc: 0.3218 - val_loss: 1.4996 - val_acc: 0.3697

Epoch 00003: val_acc did not improve from 0.37593
Epoch 4/30
2027/2027 [==============================] - 78s 39ms/step - loss: 2.0009 - acc: 0.3606 - val_loss: 2.5438 - val_acc: 0.4404

Epoch 00004: val_acc improved from 0.37593 to 0.44045, saving model to /floyd/home/m7_t1.h5
Epoch 5/30
2027/2027 [==============================] - 79s 39ms/step - loss: 1.9046 - acc: 0.3949 - val_loss: 1.6985 - val_acc:

In [12]:
%%time
# setting up model saving checkpoints
m7_t2_cp = ModelCheckpoint(filepath=f'{model_path}m7_t2.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m7_t2_history = m7_t2.fit(train_generator_2,
                                steps_per_epoch=len(X_train)//2,
                                epochs=n_epochs,
                                callbacks=[m7_t2_cp],
                                validation_data=val_generator_2,
                                validation_steps=len(X_val)//2)

# adding training history to list
model_hists.append(m7_t2_history)

Epoch 1/30
4054/4054 [==============================] - 126s 31ms/step - loss: 2.5713 - acc: 0.1806 - val_loss: 0.7337 - val_acc: 0.2938

Epoch 00001: val_acc improved from -inf to 0.29383, saving model to /floyd/home/m7_t2.h5
Epoch 2/30
4054/4054 [==============================] - 126s 31ms/step - loss: 2.2602 - acc: 0.2796 - val_loss: 1.0933 - val_acc: 0.3753

Epoch 00002: val_acc improved from 0.29383 to 0.37531, saving model to /floyd/home/m7_t2.h5
Epoch 3/30
4054/4054 [==============================] - 126s 31ms/step - loss: 2.1060 - acc: 0.3162 - val_loss: 3.3888 - val_acc: 0.3926

Epoch 00003: val_acc improved from 0.37531 to 0.39259, saving model to /floyd/home/m7_t2.h5
Epoch 4/30
4054/4054 [==============================] - 125s 31ms/step - loss: 2.0086 - acc: 0.3535 - val_loss: 2.7699 - val_acc: 0.4556

Epoch 00004: val_acc improved from 0.39259 to 0.45556, saving model to /floyd/home/m7_t2.h5
Epoch 5/30
4054/4054 [==============================] - 125s 31ms/step - loss: 1.93

In [13]:
# creating dictionary for model names, models, and histories from respective lists
models_dict = {i:[j,k] for i,j,k in zip(model_names,model_list,model_hists)}

In [15]:
(val_loss_1, val_accuracy_1) = m7_t1.evaluate(val_generator_1,verbose=1)
print(f'Val Accuracy 1: {round((val_accuracy_1*100),2)}%')
print(f'Val Loss 1: {round(val_loss_1,4)}')
print('---')
(val_loss_2, val_accuracy_2) = m7_t2.evaluate(val_generator_2,verbose=1)
print(f'Val Accuracy 2: {round((val_accuracy_2*100),2)}%')
print(f'Val Loss 2: {round(val_loss_2,4)}')
print('---')

203/203 [==============================] - 2s 10ms/step
Val Accuracy 1: 62.84%
Val Loss 1: 0.0756
---
405/405 [==============================] - 3s 7ms/step
Val Accuracy 2: 59.26%
Val Loss 2: 1.2645
---


In [16]:
# saving training histories
for key, value in models_dict.items():
    with open(f'{model_path}{key}_history', 'wb') as file_pi:
        pickle.dump(value[1].history, file_pi)
    print(f'{key}_history saved in {model_path}')

m7_t1_history saved in /floyd/home/
m7_t2_history saved in /floyd/home/
